In [1]:
from tqdm import tqdm
from PIL import Image
import numpy as np
import pandas as pd

## Calculate maximum crop from CelebA images

In [2]:
# find the quadratic bbox of maximal size (without padding) that is as centered as possible around the face
def calculate_crop(image, face_bbox):

    # find centre of bbox
    c_x = face_bbox[0] + face_bbox[2] // 2
    c_y = face_bbox[1] + face_bbox[3] // 2
    # print("center:", (c_x, c_y))


    if image.width <= image.height:
        crop_size = image.width
        left = 0
        right = crop_size
        top = c_y - crop_size // 2
        bottom = c_y + crop_size // 2

        if top < 0:
            bottom -= top
            top = 0
        if bottom > image.height:
            overlap = bottom - image.height
            top -= overlap
            bottom = image.height

    else:
        crop_size = image.height
        top = 0
        bottom = crop_size
        left = c_x - crop_size // 2
        right = c_x + crop_size // 2

        if left < 0:
            right -= left
            left = 0
        if right > image.width:
            overlap = right - image.width
            left -= overlap
            right = image.width

    return np.array((left, top, right, bottom)), crop_size

## Create and save bounding boxes of cropped images from CelebA

In [3]:
# set parameters
bb_path = "./datasets/celeba/list_bbox_celeba.txt"
img_dir = "CelebA/uncropped"
output_size = (448, 448)
new_bboxes_path = f"CelebA/augmentations/bboxes_cropped/{output_size[0]}_px_first_10k.csv"

min_index = 1 # 1
max_index = 10000 #202599


# format: (left, top, width, height)
face_bboxes = []
with open(bb_path, "r") as f:
    face_bboxes = f.readlines()

columns_result = ["Image_Name", "Left", "Top", "Right",  "Bottom"]
new_bboxes = pd.DataFrame(columns=columns_result)

# iterate through all images specified
for img_nmb in tqdm(range(min_index, max_index + 1), desc="Creating crops"):
    img_number = f"{img_nmb:06}"

    path = f"{img_dir}/{img_number}.jpg"

    image = Image.open(path)
    # image.show()
    # print("original image size:" , image.size)

    # get bbox position

    # print(face_bboxes[img_nmb + 1])
    face_bbox = face_bboxes[img_nmb + 1].split()[1:]
    if img_nmb == 101283:
        face_bbox = [70, 570, 550, 400]

    face_bbox = [int(value) for value in face_bbox]
    # print(face_bbox)
    

    # crop the image: (left, top, right, bottom)
    image_crop, crop_size = calculate_crop(image, face_bbox)
    # print("crop:", image_crop)

    scaling_factor = output_size[0] / crop_size
    face_left = face_bbox[0] - image_crop[0]
    face_right = face_left + face_bbox[2]
    face_top = face_bbox[1] - image_crop[1]
    face_bottom = face_top + face_bbox[3]
    new_face_bb = np.array((face_left, face_top, face_right, face_bottom)) * scaling_factor
    new_face_bb = np.round(new_face_bb).astype(int)

    image = image.crop(image_crop)
    image = image.resize(output_size, Image.Resampling.LANCZOS)
    # print("output image size", image.size)
    # image.show()

    image = image.crop(new_face_bb)
    new_row = {columns_result[0]: f"{img_number}.jpg", 
               columns_result[1]: new_face_bb[0],
               columns_result[2]: new_face_bb[1],
               columns_result[3]: new_face_bb[2],
               columns_result[4]: new_face_bb[3]
               }
    new_bboxes = new_bboxes.append(new_row, ignore_index=True)

    # image.show()

    # image.save(f"fairface/detected_faces_CelebA_10k_aug_samples//{img_number}.jpg")

# save new bounding boxes as csv
# print(new_bboxes)
new_bboxes.to_csv(new_bboxes_path, mode="a")
print(f"Saved bounding boxes of cropped images here: '{new_bboxes_path}'.")

Creating crops: 100%|██████████| 10000/10000 [02:16<00:00, 73.34it/s]

Saved bounding boxes of cropped images here: 'CelebA/augmentations/bboxes_cropped/448_px_first_10k.csv_last'.
